### 6개키워드 중 하나만 나오게 하는 샘플모델링

## ./crawled_img/(분위기키워드 폴더명)
 - joyful(활발한)
 - adventure(모험적인)
 - cultural(문화적인)
 - tradition(전통적인)
 - nature(자연)
 - art(예술적인)

### CNN 사용한 샘플모델링(데이터 증강X)

In [15]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 이미지 크기 설정
img_size = (224, 224) # 이미지 크키가 크면 메모리를 많이 차지해서 설정 필요

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 1000) #1000개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)


# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
150/150 [==============================] - 396s 3s/step - loss: 1.5086 - accuracy: 0.4067 - val_loss: 1.2578 - val_accuracy: 0.5050
Epoch 2/20
150/150 [==============================] - 361s 2s/step - loss: 1.1150 - accuracy: 0.5756 - val_loss: 1.0760 - val_accuracy: 0.5775
Epoch 3/20
150/150 [==============================] - 312s 2s/step - loss: 0.8595 - accuracy: 0.6825 - val_loss: 0.8565 - val_accuracy: 0.6808
Epoch 4/20
150/150 [==============================] - 313s 2s/step - loss: 0.6426 - accuracy: 0.7677 - val_loss: 0.9079 - val_accuracy: 0.6642
Epoch 5/20
150/150 [==============================] - 306s 2s/step - loss: 0.4483 - accuracy: 0.8508 - val_loss: 1.1248 - val_accuracy: 0.6517
Epoch 6/20
150/150 [==============================] - 327s 2s/step - loss: 0.2220 - accuracy: 0.9265 - val_loss: 1.1591 - val_accuracy: 0.6875
Epoch 7/20
150/150 [==============================] - 299s 2s/step - loss: 0.0966 - accuracy: 0.9710 - val_loss: 1.5454 - val_accuracy: 0.6658

In [16]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 14s 381ms/step - loss: 2.2817 - accuracy: 0.6850
테스트 손실: 2.2817
테스트 정확도: 68.50%


### 기존코드 + Dropout  -- 이미지 1000장
모델의 과적합을 방지하고 성능을 개선하기 위한 방법 중 Dropout이라는 기법을 사용하기위한 코드

Dropout은 과적합을 막기 위해 학습 과정 중 무작위로 일부 뉴런을 끄는 방법이다

아래는 Dropout을 추가한 다:

In [6]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout

# 이미지 크기 설정
img_size = (224, 224) # 이미지 크키가 크면 메모리를 많이 차지해서 설정 필요

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 1000) #1000개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)


# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout 추가
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
150/150 [==============================] - 289s 2s/step - loss: 1.7839 - accuracy: 0.3038 - val_loss: 1.4786 - val_accuracy: 0.3575
Epoch 2/20
150/150 [==============================] - 282s 2s/step - loss: 1.4079 - accuracy: 0.4338 - val_loss: 1.3247 - val_accuracy: 0.4625
Epoch 3/20
150/150 [==============================] - 277s 2s/step - loss: 1.2638 - accuracy: 0.5117 - val_loss: 1.1763 - val_accuracy: 0.5600
Epoch 4/20
150/150 [==============================] - 276s 2s/step - loss: 1.1902 - accuracy: 0.5448 - val_loss: 1.1813 - val_accuracy: 0.5558
Epoch 5/20
150/150 [==============================] - 276s 2s/step - loss: 1.1376 - accuracy: 0.5675 - val_loss: 1.1082 - val_accuracy: 0.5792
Epoch 6/20
150/150 [==============================] - 275s 2s/step - loss: 1.0895 - accuracy: 0.5831 - val_loss: 1.0739 - val_accuracy: 0.5858
Epoch 7/20
150/150 [==============================] - 273s 2s/step - loss: 1.0016 - accuracy: 0.6204 - val_loss: 1.0597 - val_accuracy: 0.5958

In [7]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 14s 375ms/step - loss: 1.3235 - accuracy: 0.6458
테스트 손실: 1.3235
테스트 정확도: 64.58%


In [8]:
from tensorflow.keras.models import save_model, load_model

# 전체 모델 저장
model.save("C:/Users/김현/Final_Project/epoch20.h5")

### epoch 50으로 변경

In [2]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout

# 이미지 크기 설정
img_size = (224, 224) # 이미지 크키가 크면 메모리를 많이 차지해서 설정 필요

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 1000) #1000개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)


# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout 추가
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
150/150 [==============================] - 287s 2s/step - loss: 1.9314 - accuracy: 0.2365 - val_loss: 1.6191 - val_accuracy: 0.3217
Epoch 2/50
150/150 [==============================] - 282s 2s/step - loss: 1.5261 - accuracy: 0.3706 - val_loss: 1.4644 - val_accuracy: 0.4633
Epoch 3/50
150/150 [==============================] - 279s 2s/step - loss: 1.3347 - accuracy: 0.4944 - val_loss: 1.3219 - val_accuracy: 0.4942
Epoch 4/50
150/150 [==============================] - 292s 2s/step - loss: 1.2721 - accuracy: 0.5167 - val_loss: 1.2786 - val_accuracy: 0.5033
Epoch 5/50
150/150 [==============================] - 439s 3s/step - loss: 1.1682 - accuracy: 0.5500 - val_loss: 1.1555 - val_accuracy: 0.5775
Epoch 6/50
150/150 [==============================] - 568s 4s/step - loss: 1.0794 - accuracy: 0.5929 - val_loss: 1.2137 - val_accuracy: 0.5450
Epoch 7/50
150/150 [==============================] - 696s 5s/step - loss: 0.9710 - accuracy: 0.6402 - val_loss: 1.0660 - val_accuracy: 0.6017

In [3]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 14s 372ms/step - loss: nan - accuracy: 0.1792
테스트 손실: nan
테스트 정확도: 17.92%


In [5]:
from tensorflow.keras.models import save_model, load_model

# 전체 모델 저장
model.save("C:/Users/김현/Final_Project/epoch50.h5")

## 기존코드 + Dropout + 데이터 증강  -- 1000장

In [21]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout

# 이미지 크기 설정
img_size = (224, 224)

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 1000) #1000개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)

# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강 설정
datagen = ImageDataGenerator(
    rotation_range=20,    # 이미지 회전 범위
    width_shift_range=0.2,    # 이미지 수평 이동 범위
    height_shift_range=0.2,   # 이미지 수직 이동 범위
    zoom_range=0.2,   # 이미지 확대/축소 범위
    horizontal_flip=True)  # 이미지 수평 뒤집기

# 이미지 데이터 증강
datagen.fit(X_train)

# 모델 학습
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          epochs=20,
          validation_data=(X_test, y_test))


Epoch 1/20
150/150 [==============================] - 400s 3s/step - loss: 1.9646 - accuracy: 0.1773 - val_loss: 1.7875 - val_accuracy: 0.1842
Epoch 2/20
150/150 [==============================] - 344s 2s/step - loss: 1.7612 - accuracy: 0.2240 - val_loss: 1.7619 - val_accuracy: 0.2192
Epoch 3/20
150/150 [==============================] - 403s 3s/step - loss: 1.7456 - accuracy: 0.2504 - val_loss: 1.7400 - val_accuracy: 0.2500
Epoch 4/20
150/150 [==============================] - 321s 2s/step - loss: 1.7343 - accuracy: 0.2438 - val_loss: 1.7519 - val_accuracy: 0.2283
Epoch 5/20
150/150 [==============================] - 315s 2s/step - loss: 1.7107 - accuracy: 0.2592 - val_loss: 1.6953 - val_accuracy: 0.2933
Epoch 6/20
150/150 [==============================] - 311s 2s/step - loss: 1.6943 - accuracy: 0.2606 - val_loss: 1.6080 - val_accuracy: 0.3358
Epoch 7/20
150/150 [==============================] - 307s 2s/step - loss: 1.6349 - accuracy: 0.3063 - val_loss: 1.5680 - val_accuracy: 0.3642

In [22]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 15s 403ms/step - loss: 1.0229 - accuracy: 0.6233
테스트 손실: 1.0229
테스트 정확도: 62.33%


### 기존코드(에포크50)에서 학습률을 낮추고, Batch Normalization 레이어를 추가하며, EarlyStopping을 적용한 것

In [9]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# 이미지 크기 설정
img_size = (224, 224) # 이미지 크키가 크면 메모리를 많이 차지해서 설정 필요

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 1000) #1000개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)


# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout 추가
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# EarlyStopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/50
150/150 [==============================] - 422s 3s/step - loss: 1.9860 - accuracy: 0.2465 - val_loss: 32.6843 - val_accuracy: 0.1617
Epoch 2/50
150/150 [==============================] - 398s 3s/step - loss: 1.6401 - accuracy: 0.2763 - val_loss: 33.0710 - val_accuracy: 0.1617
Epoch 3/50
150/150 [==============================] - 385s 3s/step - loss: 1.6097 - accuracy: 0.3117 - val_loss: 16.1516 - val_accuracy: 0.2108
Epoch 4/50
150/150 [==============================] - 383s 3s/step - loss: 1.5202 - accuracy: 0.3581 - val_loss: 9.0943 - val_accuracy: 0.3150
Epoch 5/50
150/150 [==============================] - 384s 3s/step - loss: 1.4684 - accuracy: 0.3827 - val_loss: 5.7529 - val_accuracy: 0.3667
Epoch 6/50
150/150 [==============================] - 1411s 9s/step - loss: 1.4448 - accuracy: 0.4054 - val_loss: 4.7067 - val_accuracy: 0.4183
Epoch 7/50
150/150 [==============================] - 441s 3s/step - loss: 1.3773 - accuracy: 0.4344 - val_loss: 3.4485 - val_accuracy: 0.

In [10]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 21s 556ms/step - loss: 5.4632 - accuracy: 0.5375
테스트 손실: 5.4632
테스트 정확도: 53.75%


In [11]:
from tensorflow.keras.models import save_model, load_model

# 전체 모델 저장
model.save("C:/Users/김현/Final_Project/epoch50_up.h5")

# epoch_up에다가 데이터 증강 추가

In [1]:
import os
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 크기 설정
img_size = (224, 224) # 이미지 크키가 크면 메모리를 많이 차지해서 설정 필요

# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(image_paths, label, brightness=True):
    data = []
    labels = []

    # 랜덤하게 100장만 선택
    selected_images = random.sample(image_paths, 1000) #1000개 이미지

    for img_path in selected_images:
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)

        # 이미지 명도와 채도 높이기
        if brightness:
            img_array = cv2.convertScaleAbs(img_array, alpha=1.2, beta=30)

        img_array = img_array.astype('float32') / 255.0  # 이미지를 0과 1 사이 값으로 정규화

        data.append(img_array)
        labels.append(label)

    return np.array(data), np.array(labels)


# 카테고리 리스트
categories = ["joyful", "adventure", "tradition", "nature", "cultural", "art"]
num_classes = len(categories)

# 이미지 데이터 로드 및 전처리
all_data = []
all_labels = []

for i, category in enumerate(categories):
    category_path = os.path.join("C:\\Users\\김현\\Final_Project\\crawled_img", category) # crawled_img/안에 있는 카테고리 리스트
    category_images = [os.path.join(category_path, img) for img in os.listdir(category_path)]
    
    data, labels = load_and_preprocess_data(category_images, label=i)
    all_data.extend(data)
    all_labels.extend(labels)

# 데이터 합치기 및 레이블 변환
X = np.array(all_data)
y = to_categorical(np.array(all_labels), num_classes=num_classes)

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN 모델 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))  # Dropout 추가
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout 추가
model.add(Dense(num_classes, activation='softmax'))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터 증강(Data Augmentation) 설정
datagen = ImageDataGenerator(
    rotation_range=20,        # 무작위 회전 범위
    width_shift_range=0.2,    # 무작위 수평 이동 범위
    height_shift_range=0.2,   # 무작위 수직 이동 범위
    horizontal_flip=True)     # 무작위 수평 뒤집기

# Data Augmentation 적용
datagen.fit(X_train)

# EarlyStopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델 학습
model.fit(datagen.flow(X_train, y_train, batch_size=32), 
          steps_per_epoch=len(X_train) / 32, 
          epochs=50, 
          validation_data=(X_test, y_test), 
          callbacks=[early_stopping])


Epoch 1/50
150/150 [==============================] - 608s 4s/step - loss: 1.9026 - accuracy: 0.2515 - val_loss: 27.9914 - val_accuracy: 0.1617
Epoch 2/50
150/150 [==============================] - 505s 3s/step - loss: 1.6750 - accuracy: 0.2894 - val_loss: 22.4360 - val_accuracy: 0.1642
Epoch 3/50
150/150 [==============================] - 460s 3s/step - loss: 1.6630 - accuracy: 0.2923 - val_loss: 10.3830 - val_accuracy: 0.2400
Epoch 4/50
150/150 [==============================] - 459s 3s/step - loss: 1.6528 - accuracy: 0.3081 - val_loss: 4.1308 - val_accuracy: 0.3283
Epoch 5/50
150/150 [==============================] - 450s 3s/step - loss: 1.6315 - accuracy: 0.2988 - val_loss: 3.1668 - val_accuracy: 0.3108
Epoch 6/50
150/150 [==============================] - 446s 3s/step - loss: 1.6227 - accuracy: 0.3090 - val_loss: 2.5991 - val_accuracy: 0.3025
Epoch 7/50
150/150 [==============================] - 468s 3s/step - loss: 1.5917 - accuracy: 0.3133 - val_loss: 2.3374 - val_accuracy: 0.2

In [2]:
# 테스트 세트에서 모델 평가
evaluation_results = model.evaluate(X_test, y_test)

# 평가 결과 출력
print(f'테스트 손실: {evaluation_results[0]:.4f}')
print(f'테스트 정확도: {evaluation_results[1]*100:.2f}%')


38/38 [==============================] - 24s 642ms/step - loss: 14.4922 - accuracy: 0.3225
테스트 손실: 14.4922
테스트 정확도: 32.25%


In [3]:
from tensorflow.keras.models import save_model, load_model

# 전체 모델 저장
model.save("C:/Users/김현/Final_Project/epoch50_up_data.h5")

C:\Users\김현\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
